In [1]:
import numpy as np

In [2]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

In [ ]:
%gui qt5
from PyQt5.Qt import QApplication

# start qt event loop
_instance = QApplication.instance()
if not _instance:
    _instance = QApplication([])
app = _instance

import pyqtgraph as pg

# create and and set layout
view = pg.GraphicsView()   
view.setWindowTitle('Your title')
layout = pg.GraphicsLayout()
view.setCentralItem(layout)
view.show()

# Set white graph
pg.setConfigOptions(antialias=True)
pg.setConfigOption('background', 'w')
pg.setConfigOption('foreground', 'k')

# add subplots
p0 = layout.addPlot(0,0)
p0.addLegend()
p0.plot([1,2,3,4,5], pen='b', name='p0')

p1 = layout.addPlot(1,0)
p1.addLegend()
p1.plot([2,2,2,2,], pen='r', name='p1')

p2 = layout.addPlot(1,0)
p2.addLegend(offset=(50, 0))
p2.plot([-1,0,1,1,], pen='g', name='p1.1')
p2.hideAxis('left')
p2.showAxis('right')

vb = p1.vb

def mouseMoved(evt):
    print('mouse_moved')
    '''
    pos = evt[0]  ## using signal proxy turns original arguments into a tuple
    mousePoint = vb.mapSceneToView(pos)
    print(mousePoint.x())
    '''
'''    
proxy = pg.SignalProxy(p1.scene().sigMouseMoved, rateLimit=10, slot=mouseMoved)
'''
p1.scene().sigMouseMoved.connect(mouseMoved)
    

In [ ]:
%gui qt5
from PyQt5.Qt import QApplication
import qdarkstyle
import pyqtgraph as pg

# start qt event loop
_instance = QApplication.instance()
if not _instance:
    _instance = QApplication([])
app = _instance
#app.setStyleSheet(qdarkstyle.load_stylesheet(qt_api='pyqt5'))

graphicsView = pg.GraphicsView()
graphicsView.setWindowTitle('Your title')
layout = pg.GraphicsLayout()
graphicsView.setCentralItem(layout)
graphicsView.show()

XY_plot_layout = layout.addLayout(row=0, col=0, rowspan=2, colspan=1, border=(50,50,0))
XY_plot = XY_plot_layout.addPlot()
ET_plot = layout.addPlot(row=2, col=0, rowspan=1, colspan=1)
ET_region_selection_plot = layout.addPlot(row=3, col=0, rowspan=1, colspan=1)
XY_plot.setAspectLocked(True, ratio=1.77)

ET_region = pg.LinearRegionItem(values=[1000,2000])
ET_region.setZValue(10) # Move item up 

ET_region_selection_plot.addItem(ET_region, ignoreBounds = True)
ET_plot.setAutoVisible(y=True)

data1 = 10000 + 15000 * pg.gaussianFilter(np.random.random(size=10000), 10) + 3000 * np.random.random(size=10000)
data2 = 15000 + 15000 * pg.gaussianFilter(np.random.random(size=10000), 10) + 3000 * np.random.random(size=10000)

ET_plot.plot(data1, pen="r")
ET_region_selection_plot.plot(data1, pen="r")

s4 = pg.ScatterPlotItem(
            size=3,
            pen=pg.mkPen(None),
            brush=pg.mkBrush(0, 0, 0, 20))
n = 10000
pos = np.random.normal(size=(2, n), scale=1e-9)
s4.addPoints(x=pos[0],
                y=pos[1],
                # size=(np.random.random(n) * 20.).astype(int),
                # brush=[pg.mkBrush(x) for x in np.random.randint(0, 256, (n, 3))],
                data=np.arange(n)
                )
XY_plot.addItem(s4)

def update():
    ET_region.setZValue(10)
    minX, maxX = ET_region.getRegion()
    ET_plot.setXRange(minX, maxX, padding=0)
    
ET_region.sigRegionChanged.connect(update)

#### Prototyping a multi-window tool

In [3]:
%gui qt5
from PyQt5.Qt import QApplication
import qdarkstyle
import pyqtgraph as pg

In [4]:
# start qt event loop
_instance = QApplication.instance()
if not _instance:
    _instance = QApplication([])
app = _instance

In [ ]:
eventdict = {}

In [8]:
class EventExplorer():
    def __init__(self, eventdict):
        self.ET_window = EventTimeSelector()
        self.ET_window.graphicsView.show()
        
        self.TestWindow = TestWindow()
        self.TestWindow.view.show()
                
class TestWindow():
    def __init__(self):
        self.view = pg.GraphicsView()   
        self.view.setWindowTitle('Event-Time-Selector')
        self.layout = pg.GraphicsLayout()
        self.view.setCentralItem(self.layout)
        
        # add subplots
        self.p0 = self.layout.addPlot(0,0)
        self.p0.addLegend()
        self.p0.plot([1,2,3,4,5], pen='b', name='p0')

        self.p1 = self.layout.addPlot(1,0)
        self.p1.addLegend()
        self.p1.plot([2,2,2,2,], pen='r', name='p1')

class EventTimeSelector():
    def __init__(self, eventdict):
        self.events = eventdict
        self.graphicsView = pg.GraphicsView()
        self.graphicsView.setWindowTitle('Event Time Selector')
        self.layout = pg.GraphicsLayout()
        self.graphicsView.setCentralItem(self.layout)
        #self.graphicsView.show()
        
        self.ET_plot = self.layout.addPlot(row=0, col=0)
        self.ET_region_selection_plot = self.layout.addPlot(row=1, col=0)
        
        self.ET_plot.plot(x = self.events['on_edges'], y = self.events['on_hist'], pen="b")
        self.ET_plot.plot(x = self.events['on_edges'], y = self.events['on_hist']+self.events['off_hist'], pen="r")
        self.ET_region_selection_plot.plot(x = self.events['on_edges'], y = self.events['on_hist']+self.events['off_hist'], pen="r")
        
class EventXYPlot():
    def __init__(self, eventdict):
        self.events = eventdict
        self.graphicsView = pg.GraphicsView()
        self.graphicsView.setWindowTitle('Event XY View')
        self.layout = pg.GraphicsLayout()
        self.graphicsView.setCentralItem(self.layout)
        #self.graphicsView.show()

In [9]:
eventador = EventExplorer()